# Setting things up

## Installation

In [1]:
%pip install AutoCarver[jupyter]

## Califorinia Housing Prices Data

In this example notebook, we will use the California Housing Prices dataset.

The California Housing Prices dataset is a well-known dataset in the field of machine learning and statistics. It provides information about housing districts in California and is frequently used for regression analysis and predictive modeling tasks.

Comprising housing-related metrics for various districts in California, such as median house value, median income, housing median age, average rooms, average bedrooms, population, households, and more, the California Housing Prices dataset is a valuable resource for exploring the relationships between different features and predicting the median house values (continuous regression).

In [3]:
import pandas as pd

from sklearn import datasets

# Load dataset directly from sklearn
housing = datasets.fetch_california_housing(as_frame=True)

# conversion to pandas
housing_data = housing["data"]
housing_data[housing["target_names"][0]] = housing["target"]

# Display the first few rows of the dataset
housing_data.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


## Target type and Carver selection

In [4]:
target = "MedHouseVal"

housing_data[target].describe()

count    20640.000000
mean         2.068558
std          1.153956
min          0.149990
25%          1.196000
50%          1.797000
75%          2.647250
max          5.000010
Name: MedHouseVal, dtype: float64

The target ``"MedHouseVal"`` is a continuous target of type ``float64`` used in a regression task. Hence we will use ``AutoCarver.ContinuousCarver`` and ``AutoCarver.selectors.RegressionSelector`` in following code blocks.

## Data Sampling

In [5]:
from sklearn.model_selection import train_test_split

# stratified sampling by target
train_set, dev_set = train_test_split(housing_data, test_size=0.33, random_state=42)

In [6]:
# checking target rate per dataset
train_set[target].mean(), dev_set[target].mean()

(2.0666362048018514, 2.072459655020552)

## Picking up columns to Carve

In [7]:
train_set.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
5088,0.9809,19.0,3.187726,1.129964,726.0,2.620939,33.98,-118.28,1.214
17096,4.2232,33.0,6.189696,1.086651,1015.0,2.377049,37.46,-122.23,3.637
5617,3.5488,42.0,4.821577,1.095436,1044.0,4.331950,33.79,-118.26,2.056
20060,1.6469,24.0,4.274194,1.048387,1686.0,4.532258,35.87,-119.26,0.476
895,3.9909,14.0,4.608303,1.089350,2738.0,2.471119,37.54,-121.96,2.360


In [8]:
# column data types
train_set.dtypes

MedInc         float64
HouseAge       float64
AveRooms       float64
AveBedrms      float64
Population     float64
AveOccup       float64
Latitude       float64
Longitude      float64
MedHouseVal    float64
dtype: object

All features are quantitative continuous features at the exception of ``Latitude`` and ``Longitude`` which are geographical featues (not supported by ``AutoCarver`` as is). All other features will be added to the list of ``quantitative_features``.

In [9]:
# lists of features per data type
quantitative_features = ["MedInc", "HouseAge", "AveRooms", "AveBedrms", "Population", "AveOccup"]
qualitative_features = []
ordinal_features = []

# user-specified ordering for ordinal features
values_orders = {}

# Using AutoCarver

## AutoCarver settings

### Representativness of modalities

The attribute ``min_freq`` allows one to choose the minimum frequency per basic modalities. It is used by **Discretizers**:

- For quantitative features, it defines the number of quantiles to initialy discretize the features with.

- For qualitative features, it defines the threshold under which a modality is grouped to either a default value or its closest modality.

In [10]:
min_freq = 0.05

**Tip:** should be set between ``0.01`` (slower, preciser, less robust) and ``0.2`` (faster, more robust)

### Desired number of modalities

The attribute ``max_n_mod`` allows one to choose the maximum number of modalities per carved feature. It is used by **Carvers** has the upper limit of number of modalities per consecutive combination of modalities.

In [11]:
max_n_mod = 5

**Tip:** should be set between ``3`` (faster, more robust) and ``7`` (slower, preciser, less robust)

### Association metric

The attribute ``sort_by`` allows one to choose the association metric used to sort combinations. Combinations of grouped modalities are ranked according to the specified modalities and the best ranked viable combination is returned by **Carvers**.

In [12]:
# Optional for ContinuousCarver, the implemented metric is "kruskal"
sort_by = "kruskal"

### Grouping NaNs

The attribute ``dropna`` allows one to choose whether or not ``numpy.nan`` should be grouped with another modality. If set to ``True``, **Carvers** will first find the most suitable combination of non-NaN values, and then test out all possible combinations with ``numpy.nan``.

In [13]:
dropna = False  # anyway, there are no numpy.nan in this dataset

### Optional attributes

#### Minimal frequency per carved modality

The attribute ``min_freq_mod`` allows one to choose the minimum frequency per output modality. It is used by **Carvers** in viability tests to put aside combinations that are not frequent enough in train or dev sets. By default, it is set to ``min_freq/2``.

In [14]:
min_freq_mod = None  # for 0.05,  at least 5 % of observations per output modality in train and dev sets 

#### Type of output carved features

The attribute ``output_dtype`` allows one to choose the output type:

* Use ``"float"`` for integer output (default)
* Use ``"str"`` for strin output

In [15]:
output_dtype = "float"  # "str"

## Fitting AutoCarver

* First, all quantitative features are discretized:
    1. Using ``ContinuousDiscretizer`` for quantile discretization that keeps track of over-represented values (more frequent than ``min_freq=0.05``)
    2. Using ``OrdinalDiscretizer`` for any remaining under-represented values (less frequent than ``min_freq/2=0.025``) to be grouped with its closest modality

* Second, all features are carved following this recipe:
    1. The raw distribution is printed out on provided ``train_set`` and ``dev_set``. It's the output of the discretization step
    2. Grouping modalities: all consecutive combinations of modalities are applied to ``train_set``
    3. Computing associations: the association metric (``sort_by="kruskal"``) is computed with the provided target ``train_set[target]``
    4. Combinations are sorted in descending order by association value
    5. Testing robustness: finds the first combination that checks the following:
        - Representativness of modalities on ``train_set`` and ``dev_set`` (all should be more frequent than ``min_freq_mod``)
        - Distinct target rates per consecutive modalities on ``train_set`` and ``dev_set`` 
        - No inversion of target rates between ``train_set`` and ``dev_set`` (same ordering of modalities by target rate)
    6. (Optional) If requested via ``dropna=True``, and if any, all combinations of modalities with ``numpy.nan`` are applied to ``train_set`` and steps 3. and 4. are run
    7. The carved distribution is printed out on provided ``train_set`` and ``dev_set``. It's the output of the carving step

In [16]:
from AutoCarver import ContinuousCarver

# intiating AutoCarver
auto_carver = ContinuousCarver(
    quantitative_features=quantitative_features,
    qualitative_features=qualitative_features,
    ordinal_features=ordinal_features,
    values_orders=values_orders,
    min_freq=min_freq,
    min_freq_mod=min_freq_mod,
    max_n_mod=max_n_mod,
    dropna=dropna,
    sort_by=sort_by,
    output_dtype=output_dtype,
    verbose=True,  # showing statistics
    copy=True,  # whether or not to return a copy of the input dataset
)

# fitting on training sample, a dev sample can be specified to evaluate carving robustness
train_set_processed = auto_carver.fit_transform(train_set, train_set[target], X_dev=dev_set, y_dev=dev_set[target])

------
[Discretizer] Fit Quantitative Features
---
 - [ContinuousDiscretizer] Fit ['AveBedrms', 'AveRooms', 'MedInc', 'AveOccup', 'Population', 'HouseAge']


 - [OrdinalDiscretizer] Fit ['AveBedrms', 'AveRooms', 'MedInc', 'AveOccup', 'Population', 'HouseAge']
------


------
[AutoCarver] Fit AveBedrms (1/6)
---

 - [AutoCarver] Raw distribution


,target_rate,frequency
x <= 9.400e-01,2.0684,0.0500
9.400e-01 < x <= 9.672e-01,2.0735,0.0500
9.672e-01 < x <= 9.832e-01,2.2167,0.0501
9.832e-01 < x <= 9.958e-01,2.1706,0.0499
9.958e-01 < x <= 1.007e+00,2.1310,0.0500
1.007e+00 < x <= 1.015e+00,2.2358,0.0500
1.015e+00 < x <= 1.025e+00,2.1668,0.0500
1.025e+00 < x <= 1.033e+00,2.2102,0.0500
1.033e+00 < x <= 1.041e+00,2.1295,0.0500
1.041e+00 < x <= 1.050e+00,2.1548,0.0500


Testing robustness    :   0%|          | 0/5035 [00:00<?, ?it/s]



 - [AutoCarver] Carved distribution


...

,target_rate,frequency
x <= 8.000e+00,2.1158,0.0537
8.000e+00 < x <= 1.200e+01,1.8220,0.0477
1.200e+01 < x <= 1.500e+01,1.8590,0.0613
1.500e+01 < x <= 1.600e+01,2.0358,0.0393
1.600e+01 < x <= 1.800e+01,1.9013,0.0596
1.800e+01 < x <= 2.000e+01,1.9399,0.0468
2.000e+01 < x <= 2.200e+01,2.0134,0.0404
2.200e+01 < x <= 2.500e+01,2.1055,0.0705
2.500e+01 < x <= 2.600e+01,2.0977,0.0300
2.600e+01 < x <= 2.800e+01,2.0218,0.0475


Testing robustness    :   2%|▏         | 91/4047 [00:01<01:03, 62.56it/s] 



 - [AutoCarver] Carved distribution


X distribution 
 
 
   
 target_rate 
 frequency 
 
 
 
 
 x <= 2.200e+01 
 1.9494 
 0.3486 
 
 
 2.200e+01 < x <= 2.600e+01 
 2.1032 
 0.1005 
 
 
 2.600e+01 < x <= 3.300e+01 
 2.0324 
 0.1732 
 
 
 3.300e+01 < x <= 4.500e+01 
 2.0628 
 0.2705 
 
 
 4.500e+01 < x 
 2.4785 
 0.1072 
 
 
 
 
 
 X_dev distribution 
 
 
 target_rate 
 frequency 
 
 
 
 
 1.9447 
 0.3403 
 
 
 2.1097 
 0.1058 
 
 
 2.0560 
 0.1672 
 
 
 2.0736 
 0.2798 
 
 
 2.4651 
 0.1069

------



## AutoCarver analysis

### Carving Summary

In [17]:
auto_carver.summary()

label                       content
feature    dtype                                     
AveBedrms  float      0              [x <= 1.058e+00]
           float      1  [1.058e+00 < x <= 1.100e+00]
           float      2  [1.100e+00 < x <= 1.138e+00]
           float      3  [1.138e+00 < x <= 1.273e+00]
           float      4               [1.273e+00 < x]
AveOccup   float      0              [x <= 2.067e+00]
           float      1  [2.067e+00 < x <= 2.898e+00]
           float      2  [2.898e+00 < x <= 3.425e+00]
           float      3  [3.425e+00 < x <= 3.877e+00]
           float      4               [3.877e+00 < x]
AveRooms   float      0              [x <= 5.531e+00]
           float      1  [5.531e+00 < x <= 5.860e+00]
           float      2  [5.860e+00 < x <= 6.273e+00]
           float      3  [6.273e+00 < x <= 6.542e+00]
           float      4               [6.542e+00 < x]
HouseAge   float      0              [x <= 2.200e+01]
           float      1  [2.200e+01 < x <= 2.600e+01]
           float      2  [2.600e+01 < x <= 3.300e+01]
           float      3  [3.300e+01 < x <= 4.500e+01]
           float      4               [4.500e+01 < x]
MedInc     float      0              [x <= 2.568e+00]
           float      1  [2.568e+00 < x <= 3.323e+00]
           float      2  [3.323e+00 < x <= 4.461e+00]
           float      3  [4.461e+00 < x <= 6.155e+00]
           float      4               [6.155e+00 < x]
Population float      0              [x <= 3.530e+02]
           float      1  [3.530e+02 < x <= 7.930e+02]
           float      2  [7.930e+02 < x <= 8.640e+02]
           float      3  [8.640e+02 < x <= 2.152e+03]
           float      4               [2.152e+03 < x]

* As requested with ``output_dtype="float"``, output labels are integers of ranks of modalities

* For quantitative feature ``Population``, the selected combination of modalities groups populations as follows:
    * modality ``0``: lower or equal to 353 people (``content==["x <= 3.530e+02"]``)
    * modality ``1``: greater than 353 people and lower or equal to 793 people  (``content==["3.530e+02 < x <= 7.930e+02"]``)
    * modality ``2``: greater than 793 people and lower or equal to 864 people (``content==["7.930e+02 < x <= 8.640e+02"]``)
    * modality ``3``: greater than 864 people and lower or equal to 2152 people (``content==["8.640e+02 < x <= 2.152e+03"]``)
    * modality ``4``: higher than 2152 people (``content==["2.152e+03 < x "]``)


### Detailed overview of tested combinations

In [18]:
auto_carver.history("AveRooms").head(50)

,combination,kruskal,viability,viability_message,grouping_nan
0,"[[x <= 3.441e+00], [3.441e+00 < x <= 3.794e+00...",1465.999414,None,[Raw X distribution],False
1,"[[x <= 3.441e+00, 3.441e+00 < x <= 3.794e+00, ...",1417.935973,True,[Combination robust between X and X_dev],False
2,"[[x <= 3.441e+00, 3.441e+00 < x <= 3.794e+00, ...",1417.241563,None,[Not checked],False
3,"[[x <= 3.441e+00, 3.441e+00 < x <= 3.794e+00, ...",1416.624227,None,[Not checked],False
4,"[[x <= 3.441e+00, 3.441e+00 < x <= 3.794e+00, ...",1416.389183,None,[Not checked],False
5,"[[x <= 3.441e+00, 3.441e+00 < x <= 3.794e+00, ...",1415.929817,None,[Not checked],False
6,"[[x <= 3.441e+00, 3.441e+00 < x <= 3.794e+00, ...",1414.416406,None,[Not checked],False
7,"[[x <= 3.441e+00, 3.441e+00 < x <= 3.794e+00, ...",1413.546297,None,[Not checked],False
8,"[[x <= 3.441e+00, 3.441e+00 < x <= 3.794e+00, ...",1413.104659,None,[Not checked],False
9,"[[x <= 3.441e+00, 3.441e+00 < x <= 3.794e+00, ...",1412.980977,None,[Not checked],False


In [19]:
auto_carver.history("Population")["viability_message"][2]

['X_dev: inversion of target rates per modality']

* The most associated combination of feature ``Population`` (the first tested out, where ``viability_message!=["Raw X distribution"]``) did not pass the viability tests. When looking in ``viability_message``:
    * ``"X_dev: inversion of target rates per modality"``: target rates (mean values of ``MedHouseVal`` per grouped modality) are not ranked the same between ``train_set`` and ``dev_set``

* For feature feature ``Population``, the 42nd combination is the first to pass the tests:
    - ``viability_message!=["Combination robust between X and X_dev"]``
    - Kruskal-Wallis' H with ``MedHouseVal`` is ``29.050321`` for this combination
    - Following combinations (less associated with the target) where not tested: ``viability_message==["Not checked"]``

* For all combinations ``grouping_nan==False`` means that it is not a combination in which NaNs are being groupedwith other modalities (as requested with ``dropna=False``)

## Saving and Loading AutoCarver

### Saving

All **Carvers** can safely be stored as a .json file.

In [20]:
import json

# storing as json file
with open('continuous_carver.json', 'w') as my_carver_json:
    json.dump(auto_carver.to_json(), my_carver_json)

### Loading

**Carvers** can safely be loaded from a .json file.

In [21]:
import json

from AutoCarver import load_carver

# loading json file
with open('continuous_carver.json', 'r') as my_carver_json:
    auto_carver = load_carver(json.load(my_carver_json))

## Applying AutoCarver

In [22]:
dev_set_processed = auto_carver.transform(dev_set)

In [23]:
dev_set_processed[auto_carver.features].apply(lambda u: u.value_counts(dropna=False, normalize=True))

,AveBedrms,AveRooms,MedInc,AveOccup,Population,HouseAge
0.0,0.558280,0.609219,0.255432,0.091897,0.052995,0.340282
1.0,0.200382,0.096888,0.197592,0.464181,0.208749,0.105843
2.0,0.095126,0.097328,0.257780,0.245009,0.049031,0.167205
3.0,0.096447,0.046682,0.187904,0.094686,0.539049,0.279800
4.0,0.049765,0.149883,0.101292,0.104228,0.150176,0.106870


# Feature Selection
## Selectors settings

### Features to select from

Here all features have been carved using ``ContinuousCarver``, hence all features are qualitative.

In [24]:
features = qualitative_features + quantitative_features + ordinal_features


### Number of features to select

The attribute ``n_best`` allows one to choose the number of features to be selected per data type (quantitative and qualitative).

In [25]:
n_best = 6  # here the number of features is low, ClassificationSelector will only be used to compute useful statistics

## Using Selectors

In [26]:
import importlib
import AutoCarver.selectors
importlib.reload(AutoCarver.selectors)

<module 'AutoCarver.selectors' from 'c:\\Users\\defra\\Desktop\\git\\PROJECTS\\AutoCarver\\docs\\source\\examples\\ContinuousRegression\\../../../../../AutoCarver\\AutoCarver\\selectors\\__init__.py'>

In [27]:
from AutoCarver.selectors import RegressionSelector

# select the most target associated qualitative features
feature_selector = RegressionSelector(
    qualitative_features=features,
    n_best=n_best,
    verbose=True,  # displays statistics
)
best_features = feature_selector.select(train_set_processed, train_set_processed[target])

------
[Selector] Selecting from qualitative features: ['AveBedrms', 'AveRooms', 'MedInc', 'AveOccup', 'Population', 'HouseAge']
---

 - [Selector] Association between X and y


,dtype,pct_nan,pct_mode,mode,kruskal_measure
MedInc,float64,0.0000,0.2500,0.0000,6207.6768
AveRooms,float64,0.0000,0.6000,0.0000,1417.9360
AveOccup,float64,0.0000,0.4500,1.0000,1026.3004
AveBedrms,float64,0.0000,0.5500,0.0000,346.0749
HouseAge,float64,0.0000,0.3486,0.0000,164.2102
Population,float64,0.0000,0.5498,3.0000,29.0503



 - [Selector] Association between X and y, filtered for inter-feature assocation


,dtype,pct_nan,pct_mode,mode,kruskal_measure
MedInc,float64,0.0000,0.2500,0.0000,6207.6768
AveRooms,float64,0.0000,0.6000,0.0000,1417.9360
AveOccup,float64,0.0000,0.4500,1.0000,1026.3004
AveBedrms,float64,0.0000,0.5500,0.0000,346.0749
HouseAge,float64,0.0000,0.3486,0.0000,164.2102
Population,float64,0.0000,0.5498,3.0000,29.0503



 - [Selector] Selected qualitative features: ['MedInc', 'AveRooms', 'AveOccup', 'AveBedrms', 'HouseAge', 'Population']
------



* Feature ``MedInc`` is the most associated with the target ``MedHouseVal``:
    - Kruskal-Wallis' H value is ``kruskal_measure=6207.67678``
    - It has 0 % of NaNs (``pct_nan=0.0``) 
    - Its mode, ``0``, represents 25 % of observed data (``pct_nan=0.2500``)

* Here, no feature where filtered out for there inter-feature association or over-represented values (no thresholds were set)